A ideia é enviar a transformação em excel para o bot do slack criado
https://pypi.org/project/slack-sdk/  https://pypi.org/project/kaleido/
A slack-sdk,  Ela será usada para trabalharmos diretamente com o bot, fazer as configurações, e enviar de fato o arquivo por ele;
E a kaleido, também encontrada no site do PyPi, que será usada para gerar os gráficos 


In [0]:
%pip install kaleido slack-sdk

In [0]:
#validando se instalou
import slack-sdk
from slack_Sdk import WebClient
import pyspark.pandas as ps

In [0]:
slack_token = ""
client = WebClient(token=slack_token)

In [0]:
#utilizando o padrao de nomenclatura do spark conseguimos montar um codigo para pegar o arquivo gerado
nome_arquivo = dbutils.fs.ls("dbfs:/databricks-results/prata/valores_reais/")[-1].name

In [0]:
#retorno, teremos a indicação do local "/databricks/driver". "/databricks/driver" dai podemos navegar como se fosso o linux para pegar o lugar exato do arquivo csv
path = "../../dbfs/databricks-results/prata/valores_reais/" + nome_arquivo  

In [0]:
#enviando o arquivo para o slack
enviando_arquivo_csv = client.files_upload_v2(
    channel="",  #id do canal do slack que queremos enviar a msg
    title="Arquivo no formato CSV do valor do real convertido",
    file=path,
    filename="valores_reais.csv",
    initial_comment="Segue anexo o arquivo CSV:",
)

gerando o grafico

In [0]:
df_valores_reais = ps.read_csv("dbfs:/databricks-results/prata/valores_reais/")

In [0]:
#vendo a impressao
#  df_valores_reais.plot.line(x="data", y='USD')

In [0]:
#criando pasta local (isso pq os arquivos são armazenados no cluster), esses dados são volateis então vão apagar se o cluster for reiniciado
!mkdir imagens

In [0]:
#salvando um grafico por cada moeda
for moeda in df_valores_reais.columns[1:]:
    fig = df_valores_reais.plot.line(x="data", y=moeda)
    fig.write_image(f".imagens/{moeda}.png")

enviando as imagens para o slack

In [0]:
def enviando_imagens(moeda_cotacao):
    enviando_imagens = client.files_upload_v2(
    channel="C05A04VRCUQ",  
    title="Enviando imagens de cotacoes",
    file=f"./imagens/{moeda_cotacao}.png"
)

In [0]:
for moeda in df_valores_reais.columns[1:]:
    enviando_imagens(moeda)

In [0]:
# # codigo enxuto com tratamento de erro
# # https://www.alura.com.br/artigos/tratamento-de-excecoes-no-python
# # https://www.alura.com.br/apostila-python-orientacao-a-objetos/excecoes-e-erros
# from slack_sdk import WebClient
# from slack_sdk.errors import SlackApiError

# client = WebClient(token="#token_do_bot")

# try:
#     for i in range(1, 4):
#         with open(f"./imagens/{i}.jpeg", "rb") as file:
#             response = client.files_upload_v2(
#                 channels="#seu_canal",
#                 initial_comment="Aqui estão as imagens 1, 2 e 3:",
#                 file=file.read(),
#                 filename=f"{i}.jpeg"
#             )
#             print(f"Arquivo {i}.jpeg enviado com sucesso!")
# except SlackApiError as e:
#     print("Erro ao enviar o arquivo)